In [4]:
from autoamp.pLM import ProteinLanguageModel
from transformers import AutoModelForCausalLM
from transformers import AutoTokenizer
import torch

In [13]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("nferruz/ProtGPT2")
model = AutoModelForCausalLM.from_pretrained("nferruz/ProtGPT2")

# Example input sequence (partial protein sequence)
input_sequence = "EVQLVESGGGLVQPGGSLRLSCAASGFSVSTKYMTWVRQAPGKGLEWVSVLYSGGSDYYADSVKGRFTISRDNSKNALYLQMNSLRVEDTGVYYCARDSSEVRDHPGHPGRSVGAFDIWGQGTMVTVSS"

# Tokenize the input sequence
inputs = tokenizer(input_sequence, return_tensors='pt')

# Generate the protein sequence
with torch.no_grad():
    outputs = model.generate(
        inputs['input_ids'],
        max_length=50,  # Adjust max_length based on your needs
        num_return_sequences=1,
        do_sample=True,  # Sampling is needed for generation tasks
        top_k=50,  # Adjust these parameters to control randomness
        top_p=0.95
    )

# Decode the generated token IDs to a protein sequence
generated_sequence = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(f"Input sequence: {input_sequence}")
print(f"Generated sequence: {generated_sequence}")



The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Input sequence: EVQLVESGGGLVQPGGSLRLSCAASGFSVSTKYMTWVRQAPGKGLEWVSVLYSGGSDYYADSVKGRFTISRDNSKNALYLQMNSLRVEDTGVYYCARDSSEVRDHPGHPGRSVGAFDIWGQGTMVTVSS
Generated sequence: EVQLVESGGGLVQPGGSLRLSCAASGFSVSTKYMTWVRQAPGKGLEWVSVLYSGGSDYYADSVKGRFTISRDNSKNALYLQMNSLRVEDTGVYYCARDSSEVRDHPGHPGRSVGAFDIWGQGTMVTVSS
EFFCFVSWLFW


In [23]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("nferruz/ProtGPT2")
model = AutoModelForCausalLM.from_pretrained("nferruz/ProtGPT2")

# Example wild-type protein sequence
sequence = "EVQLVESGGGLVQPGGSLRLSCAASGFSVSTKYMTWVRQAPGKGLEWVSVLYSGGSDYYADSVKGRFTISRDNSKNALYLQMNSLRVEDTGVYYCARDSSEVRDHPGHPGRSVGAFDIWGQGTMVTVSS"

# Function to predict single point mutations
def predict_mutations(sequence):
    predictions = {}
    for i in range(len(sequence)):
        # Mask the i-th position
        masked_sequence = sequence[:i] + tokenizer.mask_token + sequence[i+1:]
        inputs = tokenizer(masked_sequence, return_tensors='pt')
        
        # Generate predictions
        with torch.no_grad():
            outputs = model.generate(inputs['input_ids'], max_length=len(sequence), do_sample=True, top_k=50, top_p=0.95)
        
        # Decode predictions
        predicted_sequence = tokenizer.decode(outputs[0], skip_special_tokens=True)
        predicted_amino_acid = predicted_sequence[i]
        
        # Store the prediction
        predictions[i] = predicted_amino_acid
        
    return predictions

# Get predictions for single point mutations
mutations = predict_mutations(sequence)
print(f"Original sequence: {sequence}")
print(f"Predicted mutations: {mutations}")


TypeError: can only concatenate str (not "NoneType") to str

In [24]:
vars(tokenizer)

{'_tokenizer': <tokenizers.Tokenizer at 0x7f04f2a95430>,
 '_decode_use_source_tokenizer': False,
 'init_inputs': (),
 'init_kwargs': {'unk_token': AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
  'bos_token': AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
  'eos_token': AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
  'add_prefix_space': False,
  'name_or_path': 'nferruz/ProtGPT2'},
 'name_or_path': 'nferruz/ProtGPT2',
 '_processor_class': None,
 'model_max_length': 1000000000000000019884624838656,
 'padding_side': 'right',
 'truncation_side': 'right',
 'model_input_names': ['input_ids', 'attention_mask'],
 'clean_up_tokenization_spaces': True,
 'split_special_tokens': False,
 'deprecation_warnings': {},
 '_in_target_context_manager': False,
 'chat_template': None,
 '_bos_token': AddedToken("<|endoftext|>", r